# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="./Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

In [6]:
import os
import sys
import argparse
import logging
from collections import deque
import matplotlib.pyplot as plt
from ddpg_agent import Agent
from workspace_utils import active_session
import torch
import time
import random
import utils
from utils import Params
from utils import set_logger

%matplotlib inline

In [10]:
def ddpg(agent, params, train_mode=True):
    n_episodes = params.train_episodes if train_mode else params.test_episodes
    max_step = params.t_max
    scores_window = deque(maxlen=100)
    scores_all = []
    add_noise = True
        
    for i_episode in range(1, n_episodes+1):
        begin = time.time()

        env_info = env.reset(train_mode=train_mode)[brain_name]
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        agent.reset()
        
        for t in range(max_step): 
            actions = agent.act(states, add_noise)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations 
            rewards = env_info.rewards
            dones = env_info.local_done
            if train_mode:
                agent.step(states, actions, rewards, next_states, dones)

            states = next_states
            scores += rewards 
            if np.any(dones):
                break

        score = np.mean(scores)
        scores_window.append(score)
        score_average = np.mean(scores_window)
        scores_all.append(score)
   
        if i_episode % 50 == 0:
            print('\rEpisode {} Average score: {:.2f} Min/Max: {:.2f}/{:.2f} Time: {:.2f} - {}.{} - {}'
                .format(i_episode, score_average, np.min(scores_all), np.max(scores_all), time.time() - begin,
                        agent.batch_size, len(agent.memory), add_noise ))        

        if score_average > params.target_score * 0.75:
            add_noise = False 
                  
                    
        if score_average >= params.target_score:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\nSolve in {:d} episodes. Average score: {:.2f}'.format(i_episode, score_average))            
            break            
            
    return scores_all

In [8]:
def plot(scores):
    # plot the scores.

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.title("Ave mean scores")
    plt.savefig("chart.png", format="png")

    plt.show()

def list_params(params, json_path):

    print("\rUsing hyperparameters from {}".format(json_path))
    for keys,values in params.__dict__.items():
        print('\r\t {}\t: {}'.format(keys, values))
        

In [ ]:
rand_seed = 0
params = Params("params.json")
list_params(params, "params.json")

agent = Agent(state_size=state_size, 
              action_size=action_size,
              random_seed=rand_seed,
              params=params)

scores = ddpg(agent, params)


Using hyperparameters from params.json
	 experiment_name	: Tennis
	 model_type	: DDPG
	 num_agents	: 2
	 buffer_size	: 1000000.0
	 batch_size	: 1024
	 gamma	: 0.99
	 time_steps	: 10
	 learn_updates	: 10
	 target_tau	: 0.001
	 lr_actor	: 0.0002
	 lr_critic	: 0.0002
	 weight_decay	: 0.0
	 dropout	: 0.2
	 mean_windowsize	: 100
	 t_max	: 1000
	 target_score	: 0.5
	 train_episodes	: 2500
	 test_episodes	: 100
Episode 50 Average score: 0.00 Min/Max: -0.00/0.05 Time: 0.40 - 1024.1732 - True
Episode 100 Average score: 0.00 Min/Max: -0.00/0.15 Time: 0.48 - 1024.3458 - True
Episode 150 Average score: 0.01 Min/Max: -0.00/0.15 Time: 0.43 - 1024.5464 - True
Episode 200 Average score: 0.02 Min/Max: -0.00/0.15 Time: 0.46 - 1024.7640 - True
Episode 250 Average score: 0.02 Min/Max: -0.00/0.15 Time: 0.46 - 1024.9716 - True
Episode 300 Average score: 0.01 Min/Max: -0.00/0.15 Time: 0.49 - 1024.11728 - True
Episode 350 Average score: 0.01 Min/Max: -0.00/0.15 Time: 0.44 - 1024.13792 - True
Episode 400 Avera

In [ ]:
plot(scores)

In [ ]:
rand_seed = 0
params = Params("params.json")
list_params(params, "params.json")

agent = Agent(state_size=state_size, 
              action_size=action_size, 
              random_seed=rand_seed,
              params=params,
              actor_trained_weight_filename="checkpoint_actor.pth", 
              critic_trained_weight_filename="checkpoint_critic.pth")

scores = ddpg(agent, params, train_mode=False)

In [ ]:
plot(scores)

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```